# Домашнее задание по теме: Кластеризация

## Подготовка окружения

In [ ]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [ ]:
# !conda install matplotlib numpy scikit-learn seaborn -y

In [ ]:
# !conda install basemap matplotlib -y

In [ ]:
# !conda install -c conda-forge umap-learn -y

In [12]:
# ipympl + widget включает интерактивные функции matplotlib
 
# !conda install ipympl -y
# !conda install -c conda-forge nodejs -y
# !conda install -c conda-forge/label/gcc7 nodejs -y
# !conda install -c conda-forge/label/cf201901 nodejs -y
# !conda install -c conda-forge/label/cf202003 nodejs -y

**ВНИМАНИЕ:**

При установке `labextension` может возникнуть Warning:
```
Config option `kernel_spec_manager_class` not recognized by `InstallLabExtensionApp`.
[W 2021-07-03 10:54:07.747 LabApp] Config option `kernel_spec_manager_class` not recognized by `LabApp`.
```
Это предупреждение не должно повлиять на результат.

После это очень важно перезапустить `jupyter-lab`. Для выключения недостаточно закрыть окно, нужно воспользоваться командой `File -> Shut Down`. Затем можно привычным способом запусить `jupyter-lab`.

In [13]:
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter labextension install jupyter-matplotlib
# !jupyter nbextension enable --py widgetsnbextension

## **ВНИМАНИЕ**

- **Задания описаны кратко, только самое необходимое, в основном только идеи и подсказки. Это творческая работа.**

- Для выполнения заданий можно использовать любые удобные способы, если в комментариях к заданию не указан конкретный метод.

- Желательно использовать максимум пройденной информации или более продвинутые способы (если нашли самостоятельно).

- Желательно писать свои идеи/мысли по поводу методов/результатов.

- В строке "Подсказка" могут быть перечислены какие-то методы, которыми можно воспользоваться, но это не обязательно, так как это только один из множества вариантов (не факт, что лучший).

- Графики можно рисовать любым способом / цветом и т.п. Главное, чтобы было понятно, что там изображено.

- При выполнении задания необходимо понимать, что значит каждое слово (на английском). Если значение слова непонятно, то необходимо разобраться с его значением самостоятельно. Пожалуйста, будьте внимательны, перепроверяйте значения слов, если не уверены.

## Импорт необходимых пакетов

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

import numpy as np
import pandas as pd

from umap import UMAP
from sklearn.decomposition import PCA

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

### Загрузка данных

[Источник (Medical Cost Personal Datasets)](https://www.kaggle.com/mirichoi0218/insurance)

## Анализ данных

### Типы данных

### Пропущенные значения

### Распределение данных

## Подготовка

#### Категориальные признаки

### Масштабирование

### Полезные функции / переменные

#### Colormap

In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

# можно заменить на любые цвета (и любое нужно количество)
# см. список цветов https://matplotlib.org/stable/gallery/color/named_colors.html
# или любой RGB

cmap_3 = ListedColormap(['fuchsia', 
                         'lime', 
                         'blue'])

cmap_4 = ListedColormap(['fuchsia',
                         'chocolate',
                         'lime', 
                         'blue'])

cmap_5 = ListedColormap(['black',
                         'fuchsia',
                         'chocolate',
                         'lime', 
                         'blue'])

cmap_7 = ListedColormap(['black', 
                         'chocolate', 
                         'fuchsia', 
                         'blue',
                         'lime', 
                         'aqua', 
                         'crimson'])

cmap_22 = ListedColormap(['black',          # 1
                          'chocolate',      # 2
                          'fuchsia',        # 3
                          'blue',           # 4
                          'lime',           # 5
                          'aqua',           # 6
                          'crimson',        # 7
                          'darkorange',     # 8
                          'deeppink',       # 9
                          'dodgerblue',     # 10
                          'darkolivegreen', # 11
                          'blueviolet',     # 12
                          'olive',          # 13
                          'tomato',         # 14
                          'purple',         # 15
                          'yellow',         # 16
                          'gold',           # 17
                          'darkgreen',      # 18
                          'teal',           # 19
                          'maroon',         # 20
                          'slategray',      # 21
                          'darkslategray']) # 22

#### Scree plot

In [ ]:
def explained_variance_plot(pca_obj):
    features = range(pca_obj.n_components_)
    cumulative_sum = np.cumsum(pca_obj.explained_variance_ratio_)

    plt.xlabel('PCA features')
    plt.ylabel('variance %')
    plt.xticks(features)

    plt.bar(features, pca_obj.explained_variance_ratio_, align='center', label='Individual explained variance')
    plt.step(range(0,len(cumulative_sum)), cumulative_sum, where='mid',label='Cumulative explained variance')

    plt.legend(loc='best')
    plt.tight_layout()

#### Функция оценки силуэта

In [ ]:
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
import matplotlib.cm as cm

def silhouette_plot(X, labels):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    # 1-й подзаголовок - это силуэтный график
    # Коэффициент силуэта может варьироваться в пределах [-1, 1], 
    # но в этом примере все лежат в пределах [-0.1, 1]
    ax.set_xlim([-0.1, 1])

    # (n_clusters+1)*10 предназначен для вставки пустого пространства 
    # между силуэтами отдельных кластеров, чтобы четко их разграничить
    n_clusters = len(set(labels))
    ax.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # silhouette_score дает среднее значение для всех выборок
    # это дает представление о плотности и разделении сформированных кластеров
    silhouette_avg = silhouette_score(X, labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # оценки силуэта для каждого образца
    sample_silhouette_values = silhouette_samples(X, labels)

    y_lower = 10
    for i in range(n_clusters):
        # агрегация оценок силуэта для выборок, принадлежащих
        # кластеру  i и их сортировка
        ith_cluster_silhouette_values = sample_silhouette_values[labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.hsv(float(i) / n_clusters)
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # обозначить участки силуэта номерами кластеров в середине
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # вычислить новый y_lower для следующего участка
        y_lower = y_upper + 10  # 10 для 0 выборки

    ax.set_title("The silhouette plot for the various clusters")
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # вертикальная линия для средней оценки силуэта всех значений
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")

    # метки осей
    ax.set_yticks([])  
    ax.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

## Визуализация данных

### N-D PCA

### 3-D PCA

### 3-D UMAP

## K-means

### Выбор `n_clusters`

### Построение модели

### Анализ результатов

### Оценка силуэта (Silhouette Score)

Типичные графики силуэта представляют метку кластера на оси y, а фактический показатель силуэта — на оси x.

Silhouette Score отображает меру того, **насколько близко каждая точка в кластере находится к точкам в соседних кластерах.**

**Значение силуэта показывает, насколько объект похож на свой кластер по сравнению с другими кластерами.**

**Размер/толщина силуэтов также пропорциональны количеству образцов внутри этого кластера.**

Чем выше коэффициенты силуэта (чем ближе к +1), тем дальше образцы кластера находятся от образцов соседних кластеров.

Значение 0 указывает, что образец находится на границе принятия решения между двумя соседними кластерами или очень близко к ней.

Отрицательные значения, напротив, указывают на то, что эти образцы могли быть отнесены к неправильному кластеру.

### 3-D PCA

### 3-D UMAP

### Анализ "представителей" кластеров

### <Проверка гипотез / Дополнительные графики / Выводы>

Этот раздел необходимо назвать самостоятельно.

Можно разделить на несколько разделов. 

Написать выводы, идеи и т.п., чтобы было понятно, что пытались проанализировать кластеры.

## DBSCAN

ВНИМАНИЕ: нормально, если получится много кластеров (~20), можно попробовать сделать меньше, провести различные эксперименты с `min_samples` и `eps`.

### Выбор `min_samples`

In [ ]:
min_samples = 

### Выбор `eps`

In [ ]:
epsilon = 

### Построение модели

### Анализ результатов

### Оценка силуэта (Silhouette Score)

Типичные графики силуэта представляют метку кластера на оси y, а фактический показатель силуэта — на оси x.

Silhouette Score отображает меру того, **насколько близко каждая точка в кластере находится к точкам в соседних кластерах.**

**Значение силуэта показывает, насколько объект похож на свой кластер по сравнению с другими кластерами.**

**Размер/толщина силуэтов также пропорциональны количеству образцов внутри этого кластера.**

Чем выше коэффициенты силуэта (чем ближе к +1), тем дальше образцы кластера находятся от образцов соседних кластеров.

Значение 0 указывает, что образец находится на границе принятия решения между двумя соседними кластерами или очень близко к ней.

Отрицательные значения, напротив, указывают на то, что эти образцы могли быть отнесены к неправильному кластеру.

### 3-D PCA

### 3-D UMAP

### Анализ "представителей" кластеров

### <Проверка гипотез / Дополнительные графики / Выводы>

Этот раздел необходимо назвать самостоятельно.

Можно разделить на несколько разделов. 

Написать выводы, идеи и т.п., чтобы было понятно, что пытались проанализировать кластеры.